In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import yaml
import pandas as pd
from pathlib import Path

from utils.logging_utils import ExperimentLogger
from training_routine.trainer import Trainer
from pipeline.walkforward import WFCVGenerator
from pipeline.wf_config import WFConfig
from models.mlp import build_model
from utils.paths import CONFIG_DIR
from utils.gpu_test import gpu_test
from utils.custom_formatter import setup_logger



# -------- argparse setup --------


# setup logger
info_logger = setup_logger("Experiment", level="INFO")

# --- run GPU test once ---
gpu_test()
info_logger.info("GPU check complete.")


# -------- load config --------
cfg = yaml.safe_load(open(CONFIG_DIR / "debug.yaml"))

# -------- data + components --------
info_logger = ExperimentLogger(cfg)

# use data path if present
data_path = cfg.get("data", {}).get("df_path")

# import wfcv
wf_config = WFConfig(**cfg["walkforward"])

if data_path:
    df = pd.read_parquet(data_path)
    wf = WFCVGenerator(df_long=df, config=wf_config)
else:
    wf = WFCVGenerator(config=wf_config)

# instantiate trainer    
trainer = Trainer(cfg, info_logger)

# Only number of rows change, columns stay constant 
input_shape = cfg["walkforward"]["lags"]

max_folds = cfg["walkforward"]["max_folds"]

# -------- train per fold --------
for fold, data in enumerate(wf.folds()):
    if fold > 0:
        break  # allow running subset of folds

    # can i take this out the for loop or do i risk leakage?
    # better to leave it here so im sure weights are initialized 
    # at each fold
    #model = build_model(cfg["model"]["hparams"], input_shape)

    #trainer.fit_eval_fold(model, data, trial=0, fold=fold)


2025-10-12 23:43:57 - GPU-Test - DEBUG - ==================== Running GPU test ==================== (gpu_test.py:15)
2025-10-12 23:43:57 - GPU-Test - DEBUG - TensorFlow version: 2.18.1 (gpu_test.py:16)
2025-10-12 23:43:57 - GPU-Test - DEBUG - Visible GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')] (gpu_test.py:19)
2025-10-12 23:43:57 - GPU-Test - DEBUG - Built with CUDA?: True (gpu_test.py:22)
2025-10-12 23:43:57 - GPU-Test - DEBUG - Build info:
OrderedDict({'cpu_compiler': '/usr/lib/llvm-18/bin/clang', 'cuda_compute_capabilities': ['sm_60', 'sm_70', 'sm_80', 'sm_89', 'compute_90'], 'cuda_version': '12.5.1', 'cudnn_version': '9', 'is_cuda_build': True, 'is_rocm_build': False, 'is_tensorrt_build': False}) (gpu_test.py:25)
2025-10-12 23:43:57 - GPU-Test - DEBUG - libcuda.so.1 is loadable ✅ (gpu_test.py:29)
2025-10-12 23:43:57 - GPU-Test - DEBUG - GPU test completed successfully. (gpu_test.py:34)
2025-10-12 23:43:57 - Experiment - INFO - GPU check complete. (26284

In [34]:
data

(array([[-0.036717, -0.087444,  0.297297, ..., -0.04771 ,  0.      ,
          0.076152],
        [-0.064386, -0.092473,  0.21327 , ..., -0.004158,  0.014614,
          0.002058],
        [-0.026648,  0.      ,  0.105187, ..., -0.017875,  0.035389,
          0.007813],
        ...,
        [-0.029289,  0.030173, -0.028243, ..., -0.000947, -0.000948,
         -0.004554],
        [-0.017475, -0.01478 , -0.007374, ...,  0.003587,  0.001277,
         -0.003825],
        [ 0.00494 , -0.007647, -0.005872, ...,  0.005159,  0.028319,
         -0.004991]], shape=(365258, 20)),
 array([-0.044693, -0.004107, -0.017442, ...,  0.003622,  0.006399,
         0.0064  ], shape=(365258,)),
 array([[ 0.051454,  0.      ,  0.070213, ..., -0.013874, -0.0444  ,
          0.010899],
        [-0.006803,  0.031202,  0.00369 , ..., -0.018379,  0.017934,
         -0.023256],
        [ 0.002923,  0.025137,  0.003554, ..., -0.019115,  0.007218,
         -0.009316],
        ...,
        [ 0.040984, -0.015748,  0.01

In [35]:
wf.df_master

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,y,window
0,-0.012107,-0.062092,0.001742,-0.053913,0.056985,0.094783,-0.011914,-0.043408,0.038656,0.040453,...,0.043708,-0.024549,-0.062916,0.044234,-0.048412,-0.021463,-0.024370,0.046628,0.027844,"(0, 20)"
1,0.054099,-0.088360,-0.052815,-0.058824,0.076823,0.119710,-0.029158,-0.060067,-0.005325,0.016657,...,0.037199,0.007384,-0.092147,0.041523,-0.024363,-0.059024,-0.085645,0.054420,0.081013,"(0, 20)"
2,-0.001606,-0.033780,0.010544,-0.033498,0.013068,0.007291,-0.025613,-0.032571,0.018901,0.041159,...,-0.009346,-0.021226,-0.024096,0.015432,-0.033435,-0.006289,-0.005063,-0.003817,0.051724,"(0, 20)"
3,-0.049069,-0.030249,-0.001835,0.029412,-0.007143,0.010791,-0.003559,0.042857,0.054795,0.014610,...,0.000000,0.047541,-0.010955,0.072785,-0.008850,-0.065476,-0.070064,0.065069,0.006431,"(0, 20)"
4,-0.017335,-0.017641,-0.013468,0.019340,0.053571,0.005297,-0.007376,0.009554,0.007361,0.000000,...,-0.025346,-0.029551,-0.069428,-0.003927,0.001314,-0.028871,-0.022973,0.062241,-0.010417,"(0, 20)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3133510,0.006730,0.015812,0.006074,0.001258,0.025000,-0.009682,0.019678,-0.031193,-0.017540,-0.018873,...,-0.000789,-0.026995,0.033969,-0.023298,0.000268,0.013264,0.003438,-0.002635,0.001718,"(6268, 6288)"
3133511,0.002841,-0.000315,0.030433,0.002750,0.007617,-0.021873,0.000927,0.009369,0.005712,-0.006592,...,-0.043639,0.015960,0.024671,-0.007923,0.014416,-0.005316,-0.012437,-0.009888,-0.006833,"(6268, 6288)"
3133512,-0.003985,-0.003147,-0.022958,-0.014186,-0.004223,-0.012889,-0.031428,0.002509,0.010711,-0.036689,...,-0.044334,-0.010584,0.000897,-0.012671,0.002722,0.008533,-0.002243,-0.005974,0.002003,"(6268, 6288)"
3133513,-0.018253,0.000991,-0.000283,-0.016552,-0.021865,-0.011716,0.006944,-0.002020,-0.013525,0.005654,...,0.010047,-0.031339,-0.011300,0.007776,0.006784,0.011008,0.003460,-0.006591,-0.002705,"(6268, 6288)"
